In [3]:
import pandas as pd
import numpy as np 


In [5]:
dados = pd.read_csv("dadoss.csv", delimiter=";", decimal=",")
dados = dados[["SAPR4", "ELET6","VIVT3", "MGLU3"]]
retornos = dados.pct_change().dropna()
cov = np.array(retornos.cov())
e_r = np.array(retornos.mean()).reshape(4, 1)

In [24]:
e_r

array([[0.00046108],
       [0.00094095],
       [0.00045098],
       [0.00436355]])

In [78]:
from scipy.optimize import minimize
def minimize_vol2(target_return, er, cov):
    """
    target_ret -> W
    """
    n = er.shape[0]
    init_guess = np.repeat(1/n, n)
    bounds = ((0.0, 1.0),)*n
    weights_sum_to_1 = {'type': 'eq',
                        'fun': lambda weights: np.sum(weights) - 1
    }
    return_is_target = {'type': 'eq',
                        'args': (er,),
                        'fun': lambda weights, er: target_return - weights.T@er
    }
    
    results = minimize(lambda weights, cov: weights.T@cov@weights, init_guess,
                       args=(cov,), method="SLSQP",
                       options={"disp":False},
                       constraints=(return_is_target, weights_sum_to_1),
                       bounds=bounds
                      )
    return results.x

In [102]:
import numpy as np 

def minimize_aleatorio(e_r, cov, rp, eps = 0.000001, sim = 10000000):
    carteiras = []
    n = e_r.shape[0]
    for _ in range(sim):
        carteira = (np.random.random(n))
        carteira = carteira/sum(carteira)
        carteiras.append((carteira).reshape((n,1)))
    
    retornos = np.array([carteira.T@e_r for carteira in carteiras])
    
    filtro = (abs(retornos - rp)<eps).reshape(sim)
    
    carteiras = np.array(carteiras)
    
    carteiras = carteiras[filtro]
    
    vol = [np.sqrt(carteira.T@cov@carteira) for carteira in carteiras]
    
    index = np.argmin(vol)
    
    return carteiras[index]

In [ ]:
dados = pd.read_csv("dados_ajustados.csv", decimal= ",", delimiter =";")
dados = dados[["SPXI", "IBOV"]]
retornos = dados.pct_change().dropna()
cov = np.array(retornos.cov())
e_r = np.array(retornos.mean()).reshape(2, 1)


returns = np.linspace(min(e_r), max(e_r), 10)
carteiras_pi = np.array([minimize_aleatorio(e_r, cov, r) for r in returns])
carteiras_scipy = np.array([minimize_vol(r, e_r, cov).reshape((2, 1)) for r in returns])

vol1 = [float(np.sqrt(w.T@cov@w)) for w in carteiras_pi]
retornos = [float(w.T@e_r) for w in carteiras_pi]
dic1 = {"Vol": vol1, "r": retornos}
df1 = pd.DataFrame(dic1)
ax = df1.plot(x="Vol", y="r", marker = "x", color="goldenrod", figsize=(12,6), kind="scatter")

vol2 = [float(np.sqrt(w.T@cov@w)) for w in carteiras_scipy]
dic2 = {"Vol": vol2, "r": returns.reshape(returns.shape[0])}
df2 = pd.DataFrame(dic2)
df2.plot(x="Vol", y="r", marker = "*", color="red", ax = ax, kind="scatter")

In [100]:
carteiras_pi

array([[[0.01359072],
        [0.98640928]],

       [[0.12470141],
        [0.87529859]],

       [[0.23581285],
        [0.76418715]],

       [[0.34692435],
        [0.65307565]],

       [[0.45803555],
        [0.54196445]],

       [[0.54196454],
        [0.45803546]],

       [[0.65307635],
        [0.34692365]],

       [[0.7641873 ],
        [0.2358127 ]],

       [[0.87529777],
        [0.12470223]],

       [[0.98640959],
        [0.01359041]]])

In [101]:
carteiras_scipy

array([[[3.88793386e-09],
        [9.99999996e-01]],

       [[1.11111114e-01],
        [8.88888886e-01]],

       [[2.22222224e-01],
        [7.77777776e-01]],

       [[3.33333335e-01],
        [6.66666665e-01]],

       [[4.44444445e-01],
        [5.55555555e-01]],

       [[5.55555555e-01],
        [4.44444445e-01]],

       [[6.66666665e-01],
        [3.33333335e-01]],

       [[7.77777776e-01],
        [2.22222224e-01]],

       [[8.88888886e-01],
        [1.11111114e-01]],

       [[9.99999996e-01],
        [3.88793370e-09]]])